In [1]:
# download presidio
!pip install presidio_analyzer presidio_anonymizer
!python -m spacy download pl_core_web_lg

  Using cached presidio_analyzer-2.2.360-py3-none-any.whl.metadata (3.4 kB)
  Using cached presidio_anonymizer-2.2.360-py3-none-any.whl.metadata (8.9 kB)
  Using cached phonenumbers-9.0.20-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_arm64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp312-cp312-win_arm64.whl.metadata (41 kB)
  Using cached spacy-3.8.11-cp312-cp312-win_arm64.whl.metadata (28 kB)
  Using cached tldextract-5.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached cryptography-44.0.3.tar.gz (711 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [4 lines of output]
      Checking for Rust toolchain....
      Rust not found, installing into a temporary directory
      Python reports platform: win-arm64
      Unsupported platform: win-arm64
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> cryptography

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
c:\Users\pbuba\AppData\Local\Programs\Python\Python313\python.exe: No module named spacy


In [1]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import json
from pprint import pprint

ModuleNotFoundError: No module named 'presidio_analyzer'

# Analyze Text for PII Entities

Using Presidio Analyzer, analyze a text to identify PII entities. 
The Presidio analyzer is using pre-defined entity recognizers, and offers the option to create custom recognizers.

The following code sample will:

- Set up the Analyzer engine: load the NLP module (spaCy model by default) and other PII recognizers
- Call analyzer to get analyzed results for "PHONE_NUMBER" entity type


In [ ]:
text_to_anonymize = "His name is Mr. Jones and his phone number is 212-555-5555"

In [ ]:
analyzer = AnalyzerEngine()
analyzer_results = analyzer.analyze(text=text_to_anonymize, entities=["PHONE_NUMBER"], language='en')

print(analyzer_results)

## Create Custom PII Entity Recognizers

Presidio Analyzer comes with a pre-defined set of entity recognizers. It also allows adding new recognizers without changing the analyzer base code, **by creating custom recognizers**. 
In the following example, we will create two new recognizers of type `PatternRecognizer` to identify titles and pronouns in the analyzed text.
A `PatternRecognizer` is a PII entity recognizer which uses regular expressions or deny-lists.

The following code sample will:
- Create custom recognizers
- Add the new custom recognizers to the analyzer
- Call analyzer to get results from the new recognizers

In [ ]:
titles_recognizer = PatternRecognizer(supported_entity="TITLE",
                                      deny_list=["Mr.","Mrs.","Miss"])

pronoun_recognizer = PatternRecognizer(supported_entity="PRONOUN",
                                       deny_list=["he", "He", "his", "His", "she", "She", "hers", "Hers"])

analyzer.registry.add_recognizer(titles_recognizer)
analyzer.registry.add_recognizer(pronoun_recognizer)

analyzer_results = analyzer.analyze(text=text_to_anonymize,
                            entities=["TITLE", "PRONOUN"],
                            language="en")
print(analyzer_results)


Call Presidio Analyzer and get analyzed results with all the configured recognizers - default and new custom recognizers

In [ ]:
analyzer_results = analyzer.analyze(text=text_to_anonymize, language='en')

analyzer_results

# Anonymize Text with Identified PII Entities

<br>Presidio Anonymizer iterates over the Presidio Analyzer result, and provides anonymization capabilities for the identified text.
<br>The anonymizer provides 5 types of anonymizers - replace, redact, mask, hash and encrypt. The default is **replace**

<br>The following code sample will:
<ol>
<li>Setup the anonymizer engine </li>
<li>Create an anonymizer request - text to anonymize, list of anonymizers to apply and the results from the analyzer request</li>
<li>Anonymize the text</li>
</ol>

In [ ]:
anonymizer = AnonymizerEngine()

anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize,
    analyzer_results=analyzer_results,    
    operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}), 
                        "PHONE_NUMBER": OperatorConfig("mask", {"type": "mask", "masking_char" : "*", "chars_to_mask" : 12, "from_end" : True}),
                        "TITLE": OperatorConfig("redact", {})}
)

print(f"text: {anonymized_results.text}")
print("detailed response:")

pprint(json.loads(anonymized_results.to_json()))